# Yinelemeli Sinir Ağları (Recurrent Neural Networks)(RNN)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

**RNN:** Tekrarlayan sinir ağları . Zaman Serilerini kullanır 

### 1)Veri Setinin Oluşturulması

In [2]:
def generate_data(seq_length = 50 , num_samples = 1000 ):
    x = np.linspace(0,100,num_samples)
    y = np.sin(x)
    sequence = [] # Giriş dizilerini saklamak için
    targets = [] # Hedef değerlerini saklamak için

    for i in range(len(x) - seq_length):
        sequence.append(y[i:i+seq_length]) # İnput
        targets.append(y[i+seq_length])

    return np.array(sequence) , np.array(targets)

In [3]:
sequence , targets = generate_data()

In [4]:
sequence[0:3]

array([[ 0.        ,  0.09993302,  0.19886554,  0.29580708,  0.3897871 ,
         0.47986471,  0.56513807,  0.64475345,  0.71791378,  0.7838866 ,
         0.84201141,  0.89170627,  0.93247368,  0.96390547,  0.98568695,
         0.99760007,  0.99952556,  0.99144412,  0.97343668,  0.94568351,
         0.90846248,  0.86214622,  0.80719845,  0.74416927,  0.67368972,
         0.59646541,  0.51326949,  0.42493488,  0.33234597,  0.23642973,
         0.13814643,  0.03848005, -0.06157158, -0.16100677, -0.25883002,
        -0.35406195, -0.44574912, -0.53297361, -0.61486214, -0.69059488,
        -0.75941361, -0.82062935, -0.87362922, -0.91788261, -0.95294646,
        -0.97846973, -0.99419689, -0.99997048, -0.9957327 , -0.98152597],
       [ 0.09993302,  0.19886554,  0.29580708,  0.3897871 ,  0.47986471,
         0.56513807,  0.64475345,  0.71791378,  0.7838866 ,  0.84201141,
         0.89170627,  0.93247368,  0.96390547,  0.98568695,  0.99760007,
         0.99952556,  0.99144412,  0.97343668,  0.

### 2)Modelin Oluşturulması

In [5]:
class RNN(nn.Module):
    def __init__(self,input_size , hidden_size , output_size , num_layers = 1):
        """
            RNN -> Linear (Output)
        """
        super(RNN,self).__init__()
        # Input Size : Giriş Boyutu
        # Hidden Size : RNN Gizli Katman (CELL) sayısı
        # Num Layers : RNN Layer sayısı
        self.rnn = nn.RNN(input_size , hidden_size , num_layers , batch_first = True) # RNN - Layer
        # Output Size : Çıktı boyutu 
        self.fc = nn.Linear(hidden_size , output_size) # Fully connected layer : output
    
    def forward(self , x):
        out , _ = self.rnn(x) # RNN'e girdiyi ver çıktıyı al
        out = self.fc(out[:,-1,:]) # Son zaman adımındaki çıktıyı al ve fc layer'a bağla
        return out

### 3)Modelin Eğitimi

In [6]:
seq_length = 50 # İnput dizisinin boyutu
input_size = 1 # İnput dizisinin boyutu
hidden_size = 16 # RNN'in gizli katmandaki düğüm sayısı
output_size = 1 # Output boyutu ya da tahmin edilen değer
num_layers = 1 # RNN katman sayısı
epochs = 20 # Modelin kaç kez tüm veri üzerinde eğitileceği
batch_size = 32 # Her bir eğitim adımında kaç örneğin kullanılacağı
learning_rate = 0.001 # Optimizasyon algoritması için öğrenme oranı hızı

x , y = generate_data(seq_length)
x = torch.tensor(x , dtype = torch.float32).unsqueeze(-1) # PyTorch tensore çevir ve boyut ekle
y = torch.tensor(y , dtype = torch.float32).unsqueeze(-1) # PyTorch tensore çevir ve boyut ekle

dataset = torch.utils.data.TensorDataset(x,y) # PyTorch dataset oluşturma
data_loader = torch.utils.data.DataLoader(dataset , batch_size = batch_size , shuffle = True ) # Veri yükleyici oluştur

**Modelin Tanımlanması**

In [7]:
model = RNN(input_size , hidden_size , output_size , num_layers)
criterion = nn.MSELoss() # Min square error - ortalama kare hata
optimizer = torch.optim.Adam(model.parameters() , lr = learning_rate) # Optimization = adaptive

for epoch in range(epochs):
    for batch_x , batch_y in data_loader:
        optimizer.zero_grad() # Gradyanları sıfırla
        pred_y = model(batch_x) # Modelin tahmini al
        loss = criterion(pred_y,batch_y) # Modelin tahmini ile gerçek olanı karşılaştır ve loss hesapla
        loss.backward() # Geri yayılım ile gradyanları hesapla
        optimizer.step() # Ağırlıkları güncelle
    print(f"Epoch : {epoch + 1} / {epochs} , Loss :{loss.item():.4f}")

Epoch : 1 / 20 , Loss :0.2557
Epoch : 2 / 20 , Loss :0.0789
Epoch : 3 / 20 , Loss :0.0423
Epoch : 4 / 20 , Loss :0.0243
Epoch : 5 / 20 , Loss :0.0150
Epoch : 6 / 20 , Loss :0.0100
Epoch : 7 / 20 , Loss :0.0075
Epoch : 8 / 20 , Loss :0.0035
Epoch : 9 / 20 , Loss :0.0022
Epoch : 10 / 20 , Loss :0.0017
Epoch : 11 / 20 , Loss :0.0010
Epoch : 12 / 20 , Loss :0.0013
Epoch : 13 / 20 , Loss :0.0010
Epoch : 14 / 20 , Loss :0.0009
Epoch : 15 / 20 , Loss :0.0009
Epoch : 16 / 20 , Loss :0.0006
Epoch : 17 / 20 , Loss :0.0005
Epoch : 18 / 20 , Loss :0.0007
Epoch : 19 / 20 , Loss :0.0004
Epoch : 20 / 20 , Loss :0.0004


### 4)Modelin Test Edilmesi

**Test için veri oluştur**

In [8]:
x_test = np.linspace(100,110,seq_length).reshape(1,-1) # İlk test verisi
y_test = np.sin(x_test) # Test verisinin gerçek değeri 

x_test2 = np.linspace(120,130,seq_length).reshape(1,-1) # İkinci test verisi
y_test2 = np.sin(x_test2) # İkinci test verisi gerçek değerleri

# From numpy to tensor
x_test = torch.tensor(y_test, dtype = torch.float32 ).unsqueeze(-1)
x_test2 = torch.tensor(y_test2, dtype = torch.float32 ).unsqueeze(-1)

**Modeli Kullanarak Predict İşlemi**

In [9]:
model.eval()
prediction1 = model(x_test).detach().numpy()
prediction2 = model(x_test2).detach().numpy()

**Sonuçları Görselleştirme**

In [ ]:
plt.figure()
plt.plot(np.linspace(0,100,len(y)),y,marker = "o", label = "Training Dataset")

plt.plot(x_test.numpy().flatten(), marker = "o" , label = "Test 1")
plt.plot(x_test2.numpy().flatten(), marker = "o" , label = "Test 2")

plt.plot(np.arange(seq_length, seq_length + 1) , prediction1.flatten() , "ro" , label = "Prediction 1")
plt.plot(np.arange(seq_length, seq_length + 1) , prediction2.flatten() , "ro" , label = "Prediction 2")

plt.legend()
plt.show()